### - Sobhan Moradian Daghigh
#### - 11/25/2021
#### 
#### - ML - EX01 - Q1
##### 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import seaborn as sns
import import_ipynb
from sklearn import preprocessing

#### Reading the dataset

In [ ]:
dataset = pd.read_excel('./ML-HW01/dataset2/CSM_dataset.xlsx')
dataset.head()

In [ ]:
dataset.info()

#### Part A:

In [ ]:
dataset.isnull().sum()

#### So we have missing values for cols 'Budget', 'Screens' and 'Aggregate Followers'

#### But before trying to handle these, Im wanna see the correlation matrix to determine how to fill in the missing values.

In [ ]:
dataset.corr().style.background_gradient(cmap='coolwarm')

#### OK lets do filling

#### Screens:

In [ ]:
dataset['C2D_Gross'] = pd.cut(dataset.Gross, bins=[0, 30000000, 100000000, 1000000000],labels=['NotGood', 'Good', 'Awesome'])
groups = dataset.groupby(by=['C2D_Gross'])
groups.first()

In [ ]:
for grp, pdf in groups:
    lst = dataset.loc[(dataset.C2D_Gross == grp) & (dataset.Screens.notnull())]['Screens'].tolist()
    if lst:
        mean = np.mean(lst)
        dataset.loc[(dataset.C2D_Gross == grp) & (dataset.Screens.isnull()), ['Screens']] = round(mean)

In [ ]:
dataset.isnull().sum()

#### Budget:

In [ ]:
groups = dataset.groupby(by=['Genre'])
groups.first()

In [ ]:
for grp, pdf in groups:
    lst = dataset.loc[(dataset.Genre == grp) & (dataset.Budget.notnull())]['Budget'].tolist()
    if lst:
        mean = np.mean(lst)
        dataset.loc[(dataset.Genre == grp) & (dataset.Budget.isnull()), ['Budget']] = round(mean)

In [ ]:
dataset.isnull().sum()

#### Aggregate Followers:

In [ ]:
groups = dataset.groupby(by=['C2D_Gross', 'Genre'])
groups.first()

In [ ]:
for grp, pdf in groups:
    lst = dataset.loc[(dataset.C2D_Gross == grp[0]) & (dataset.Genre == grp[1]) & (dataset.loc[:, 'Aggregate Followers'].notnull())]['Aggregate Followers'].tolist()
    if lst:
        mean = np.mean(lst)
        dataset.loc[(dataset.C2D_Gross == grp[0]) & (dataset.Genre == grp[1]) & (dataset.loc[:, 'Aggregate Followers'].isnull()), ['Aggregate Followers']] = round(mean)
        
    lst = dataset.loc[(dataset.C2D_Gross == grp[0]) & (dataset.loc[:, 'Aggregate Followers'].notnull())]['Aggregate Followers'].tolist()
    if lst:
        mean = np.mean(lst)
        dataset.loc[(dataset.C2D_Gross == grp[0]) & (dataset.loc[:, 'Aggregate Followers'].isnull()), ['Aggregate Followers']] = round(mean)

In [ ]:
dataset.pop('C2D_Gross')
dataset.isnull().sum()

#### All the missing values handled.

#### Part B:

In [ ]:
# sns.set(style="ticks", color_codes=True)    
# g = sns.pairplot(dataset)
# plt.show()

#### Part D:

In [ ]:
import Ex01_Q1 as Q1

In [ ]:
dataset['Ratings'] = dataset.pop('Ratings')
dataset.insert(loc=1, column='Bias', value=[1] * dataset.shape[0])

#### Normalizing

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(dataset.iloc[:, 2:].values)
scaled_dataset = pd.DataFrame(x_scaled, columns=['Year','Genre', 'Gross', 'Budget', 'Screens', 'Sequel', 'Sentiment', 'Views', 'Likes', 'Dislikes', 'Comments', 'Aggregate Followers', 'Ratings'])
scaled_dataset.insert(loc=0, column='Movie', value=dataset.iloc[:, 0])
scaled_dataset.insert(loc=1, column='Bias', value=dataset.iloc[:, 1])

scaled_dataset.iloc[:, 2:-1] += 0.0005

scaled_dataset.head()

In [ ]:
# scaled_dataset.insert(loc=5, column='Gross2', value=np.power(scaled_dataset.iloc[:, 4], 2))
# scaled_dataset.insert(loc=7, column='Budget2', value=np.multiply(scaled_dataset.iloc[:, 6], 2))

# scaled_dataset.head()

#### Split data into 80% of train and 20% of test.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(scaled_dataset.iloc[:,:-1], scaled_dataset.iloc[:, -1], train_size=0.8, shuffle=True)

In [ ]:
train = pd.concat([x_train, y_train], axis=1)
test = pd.concat([x_test, y_test], axis=1)

In [ ]:
train.head()

In [ ]:
# Data count
m = train.shape[0]

degree = train.shape[1] - 3 # 12 features
epoch = 5000

coeffs, mr_lr_ep = Q1.gd(epoch=epoch, degree=degree, df_train=train.iloc[:, 1:], df_test=test.iloc[:, 1:], 
                  size=m, alpha=0.06, decay=0.0001) #decay:0.00003
train_pred = Q1.predY_cal(coeffs, train.iloc[:, 1:], degree=degree)
test_pred = Q1.predY_cal(coeffs, test.iloc[:, 1:], degree=degree)

tr_RMSE = mean_squared_error(train.iloc[:, -1], train_pred, squared=False)
ts_RMSE = mean_squared_error(test.iloc[:, -1], test_pred, squared=False)

tr_R2Score = r2_score(train.iloc[:, -1], train_pred)
ts_R2Score = r2_score(test.iloc[:, -1], test_pred)

print('Test RMSE: {:.2f} , Train RMSE: {:.2f}'.format(ts_RMSE, tr_RMSE))
print('Test R2Score: {:.2f} , Train R2Score: {:.2f}'.format(ts_R2Score, tr_R2Score))  

In [ ]:
# without Gross2
# Test RMSE: 0.16 , Train RMSE: 0.15
# Test R2Score: 0.25 , Train R2Score: 0.24

In [ ]:
## Test
# decays = [0.01, 0.001, 0.0001, 0.00001, 0.000001]
# alphas = [0.6, 0.7, 0.8, 0.9, 1]

In [ ]:
# for alpha in alphas:
#     for decay in decays:
#         m = train.shape[0]

#         degree = train.shape[1] - 3 # 12 features
#         epoch = 1000

#         coeffs, mr_lr_ep = Q1.gd(epoch=epoch, degree=degree, df_train=train.iloc[:, 1:], df_test=test.iloc[:, 1:], 
#                           size=m, alpha=alpha, decay=decay) #decay:0.00003
#         train_pred = Q1.predY_cal(coeffs, train.iloc[:, 1:], degree=degree)
#         test_pred = Q1.predY_cal(coeffs, test.iloc[:, 1:], degree=degree)
        
#         tr_RMSE = mean_squared_error(train.iloc[:, -1], train_pred, squared=False)
#         ts_RMSE = mean_squared_error(test.iloc[:, -1], test_pred, squared=False)

#         tr_R2Score = r2_score(train.iloc[:, -1], train_pred)
#         ts_R2Score = r2_score(test.iloc[:, -1], test_pred)

#         print(alpha, decay)
#         print('Test RMSE: {:.2f} , Train RMSE: {:.2f}'.format(ts_RMSE, tr_RMSE))
#         print('Test R2Score: {:.2f} , Train R2Score: {:.2f}'.format(ts_R2Score, tr_R2Score)) 
#         print('\n\n')

In [ ]:
fig, axs = plt.subplots(3, figsize=(10, 15))

axs[0].set_title('Train [degree: {} , epoch {}]'.format(degree, epoch))
axs[0].plot(mr_lr_ep[2], mr_lr_ep[0][0][0], 'g', label='MSE')
axs[0].plot(mr_lr_ep[2], mr_lr_ep[0][0][1], 'r', label='RMSE')
axs[0].plot(mr_lr_ep[2], mr_lr_ep[0][0][2], 'b', label='MAE')
axs[0].legend()        

axs[1].set_title('Test [degree: {} , epoch {}]'.format(degree, epoch))
axs[1].plot(mr_lr_ep[2], mr_lr_ep[0][1][0], 'g', label='MSE')
axs[1].plot(mr_lr_ep[2], mr_lr_ep[0][1][1], 'r', label='RMSE')
axs[1].plot(mr_lr_ep[2], mr_lr_ep[0][1][2], 'b', label='MAE')
axs[1].legend()        

axs[2].set_title('epoch {}'.format(epoch))
axs[2].plot(mr_lr_ep[2], mr_lr_ep[1], 'c', label='Step size')
axs[2].legend()


In [ ]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

new_train = train.copy()
new_test = test.copy()

new_train.pop('Year')
new_test.pop('Year')

new_train.pop('Comments')
new_test.pop('Comments')

# new_train.pop('Gross')
# new_test.pop('Gross')

# new_train.pop('Budget')
# new_test.pop('Budget')

# new_train.pop('Likes')
# new_test.pop('Likes')

# new_train.pop('Screens')
# new_test.pop('Screens')

# new_train.pop('Genre')
# new_test.pop('Genre')

# new_train.pop('Sequel')
# new_test.pop('Sequel')

# new_train.pop('Sentiment')
# new_test.pop('Sentiment')

# new_train.pop('Views')
# new_test.pop('Views')

# new_train.pop('Dislikes')
# new_test.pop('Dislikes')

# new_train.pop('Aggregate Followers')
# new_test.pop('Aggregate Followers')

# Genre	Gross	Budget	Screens	Sequel	Sentiment	Views	Likes	Dislikes	Comments	Aggregate Followers
# Data count
m = train.shape[0]

degree = new_train.shape[1] - 3 # 3: movie + bias + ratings
epoch = 5000

In [ ]:
degree

In [ ]:
coeffs

In [ ]:
new_train.iloc[:, 1:]

In [ ]:
coeffs, mr_lr_ep = Q1.gd(epoch=epoch, degree=degree, df_train=new_train.iloc[:, 1:], df_test=new_test.iloc[:, 1:], 
                  size=m, alpha=0.001, decay=0.0000003)
train_pred = Q1.predY_cal(coeffs, new_train.iloc[:, 1:], degree=degree)
test_pred = Q1.predY_cal(coeffs, new_test.iloc[:, 1:], degree=degree)

tr_RMSE = mean_squared_error(new_train.iloc[:, -1], train_pred, squared=False)
ts_RMSE = mean_squared_error(new_test.iloc[:, -1], test_pred, squared=False)

tr_R2Score = r2_score(new_train.iloc[:, -1], train_pred)
ts_R2Score = r2_score(new_test.iloc[:, -1], test_pred)

print('Test RMSE: {:.2f} , Train RMSE: {:.2f}'.format(ts_RMSE, tr_RMSE)) 
print('Test R2Score: {:.2f} , Train R2Score: {:.2f}'.format(ts_R2Score, tr_R2Score))  

In [ ]:
fig, axs = plt.subplots(3, figsize=(10, 15))

axs[0].set_title('Train [degree: {} , epoch {}]'.format(degree, epoch))
axs[0].plot(mr_lr_ep[2], mr_lr_ep[0][0][0], 'g', label='MSE')
axs[0].plot(mr_lr_ep[2], mr_lr_ep[0][0][1], 'r', label='RMSE')
axs[0].plot(mr_lr_ep[2], mr_lr_ep[0][0][2], 'b', label='MAE')
axs[0].legend()        

axs[1].set_title('Test [degree: {} , epoch {}]'.format(degree, epoch))
axs[1].plot(mr_lr_ep[2], mr_lr_ep[0][1][0], 'g', label='MSE')
axs[1].plot(mr_lr_ep[2], mr_lr_ep[0][1][1], 'r', label='RMSE')
axs[1].plot(mr_lr_ep[2], mr_lr_ep[0][1][2], 'b', label='MAE')
axs[1].legend()        

axs[2].set_title('epoch {}'.format(epoch))
axs[2].plot(mr_lr_ep[2], mr_lr_ep[1], 'c', label='Learning rate')
axs[2].legend()

In [ ]:
# from sklearn.linear_model import LinearRegression

# reg = LinearRegression().fit(new_train.iloc[:, 1:-1], new_train.iloc[:, -1])

# print('R2: ', reg.score(new_train.iloc[:, 1:-1], new_train.iloc[:, -1]))
# pred = Q1.predY_cal(reg.coef_, train.iloc[:, 1:], degree=12)
# print('MSE: ', mean_squared_error(train.iloc[:, -1], pred))

In [ ]:
# reg.coef_